**CNN-LSTM**

cross validation and training of CRNN network. Change directory paths to relevant ones

In [ ]:
#if carrying out on google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:

import pandas as pd
import numpy as np
import os


import torch
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
from sklearn.model_selection import StratifiedShuffleSplit
import pickle
import torch.nn as nn


In [2]:
# directory storing the DFC dataset
data_dir = 'dfc_cc200'

In [3]:
df_p = 'phenotype_files/pheno_nn.csv'

In [4]:
df_raw = pd.read_csv(df_p)

In [5]:
df_raw.head()

,SITE_ID,X,SUB_ID,FILE_ID,AGE_AT_SCAN,SEX,DSM_IV_TR,DX_GROUP
0,PITT,1,50002,Pitt_0050002,16.77,1,1,1
1,PITT,2,50003,Pitt_0050003,24.45,1,1,1
2,PITT,3,50004,Pitt_0050004,19.09,1,1,1
3,PITT,4,50005,Pitt_0050005,13.73,2,1,1
4,PITT,5,50006,Pitt_0050006,13.37,1,1,1


In [6]:
# binarize target column and combine with File_id to form dataset annotation file
df = pd.DataFrame({'FILE_ID': df_raw.FILE_ID.values,'TARGET':[1 if i == 1 else 0 for i in df_raw.DSM_IV_TR.values]})

In [ ]:
class DfcDataset(Dataset):

    def __init__(self,df,dfc_dir):
        """
        :param df: pandas dataframe containing subject ids in column `FILE_ID` and target labels in column `TARGET`
        :param dfc_dir: path to subdirectory containing the pickle files
        """
        self.df_main = df
        self.dfc_dir = dfc_dir
        # get paths to each DFC matrix
        self.paths = [os.path.join(dfc_dir,f + '_dfc.npy') for f in df.FILE_ID]

    def __getitem__(self, idx):
        """
        :param idx: index
        :return: 4D tensor with 2 spatial dimensions and 1 temporal [N,1,90,200,200], and label
        """

        dfc_array = np.load(self.paths[idx])# load array
        dfc_tensor = [torch.from_numpy(npDfc).type(torch.float) for npDfc in dfc_array] #take each list in dfc_array and turn to torch tensor of type torch.float, otherwise result is float(64)

        dfc_stack = torch.stack(dfc_tensor) # make tensor stack of each dfc array

        label = self.df_main.loc[idx, 'TARGET'] # class label of idx as np array
        label_tensor = torch.tensor(label)
        label_tensor = label_tensor.to(torch.float32)

        return torch.unsqueeze(dfc_stack, dim= 0),torch.unsqueeze(label_tensor, dim = -1) # turn torch stack to 4D tensor for input into conv3d (channel,depth,height,width), return label as tensor of shape [batch size, 1]

    def __len__(self):
        """
        :return: length of dataset
        """
        return len(self.df_main)

In [ ]:
class crnn(nn.Module):
    def __init__(self):
        super(crnn, self).__init__()

        #Convolutional layers
        self.seq_cnn = nn.Sequential(
            nn.Conv3d(in_channels=1,out_channels=8, kernel_size=(2,200,1),stride=(1,1,1), padding_mode = 'reflect'),
            nn.GELU(),
            nn.Conv3d(in_channels=8,out_channels=16, kernel_size=(2,1,200),stride=(1,1,1), padding_mode = 'reflect'),
            nn.GELU(),
            nn.Conv3d(in_channels=16,out_channels=32, kernel_size=(8,1,1),stride=(2,1,1), padding_mode = 'reflect'),
            nn.GELU(),
            nn.BatchNorm3d(32)
        )



        #lstm layer
        self.lstm1 = nn.GRU(input_size=32, hidden_size=64, num_layers=1, batch_first = True)

        # Fully connected layer
        self.seq_dense = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(16,1),
            nn.Sigmoid()
        )


    def forward(self, x):

        x = self.seq_cnn(x)


        x = torch.flatten(x,start_dim=2,end_dim=4) # reduce dimensionality for LSTM layer, to 3D tensor
        x = x.permute(0,2,1) # transpose to make tensor of size [batch_size, sequence length, feature number]

        x,_ = self.lstm1(x)

        x = x[:,-1,:] # take output of last LSTM cell

        x = self.seq_dense(x)

        return x

In [ ]:
# set device to GPU if gpu is absent training will be done on CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
def train_one_epoch(model,loss_fn, optimiser ,train_loader, device):
  running_loss = 0
  epoch_accuracy = []
  for j, (x_train,y) in enumerate(train_loader):
    optimiser.zero_grad() # zero the gradient at each epoch start
    y = y.to(device) # send y to cuda
    x_train = x_train.to(device)
    prediction = model.forward(x_train)
    loss = loss_fn(prediction,y) # loss

    accuracy = (torch.round(prediction) == y).float().mean() # calculate accuracy for each mini-batch  take prediction tensor, reshape to 1d detach from computational graph turn to numpy array, round and see if rounded number is equal to label, find mean of this boolean array, this is the accuracy

    running_loss += loss.item() # get epoch loss
    epoch_accuracy.append(accuracy.item())
                

    loss.backward() # backward propgation
    optimiser.step()
    
    running_loss += loss.item() # get epoch loss
    epoch_accuracy.append(accuracy.item())

  return (running_loss, np.mean(epoch_accuracy))



In [ ]:
def validate_one_epoch(model,loss_fn, test_loader, device):
  test_loss_run = 0
  test_acc_epoch = []
  for j, (x_test,y_test) in enumerate(test_loader):
    y_test = y_test.to(device)
    x_test= x_test.to(device)
    test_pred = model.forward(x_test)
    test_loss = loss_fn(test_pred,y_test) # loss

    test_acc = (torch.round(test_pred) == y_test).float().mean() # calculate accuracy for each mini-batch  take prediction tensor, reshape to 1d detach from computational graph turn to numpy array, round and see if rounded number is equal to label, find mean of this boolean array, this is the accuracy

    test_loss_run += test_loss.item()
    test_acc_epoch.append(test_acc.item())
  
  return (test_loss_run, np.mean(test_acc_epoch))

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
cv = StratifiedShuffleSplit(n_splits=10)
scores = []
cv_metrics = []
for train, test in cv.split(df['FILE_ID'], df['TARGET']):
    model = crnn()
    loss_fn = nn.BCELoss()
    optimiser =  optim.AdamW(model.parameters(), lr=0.0001, weight_decay = 1e-2)

    train_df = df.iloc[train, :]
    train_df.reset_index(drop=True, inplace=True)
    train_data = DfcDataset(train_df,data_dir)
    train_dataloader = DataLoader(train_data, batch_size= 16,
                                        shuffle=True,
                                        num_workers=0)
    test_df = df.iloc[test, :]
    test_df.reset_index(drop=True, inplace=True)
    test_data = DfcDataset(test_df,data_dir)
    test_dataloader = DataLoader(test_data, batch_size= 16,
                                        shuffle=True,
                                        num_workers=0)

    model.to(device)

    train_loss_history = []
    train_acc_history = []
    test_loss_history = []
    test_acc_history = []
    for i in range(20):

      model.train()
      train_loss, train_acc = train_one_epoch(model,loss_fn,optimiser,train_dataloader,device)


      train_loss_history.append(train_loss)
      train_acc_history.append(train_acc)

      model.eval()
      test_loss, test_acc = validate_one_epoch(model,loss_fn, test_dataloader,device)

      test_loss_history.append(test_loss)
      test_acc_history.append(test_acc)

  

      
      print(f'Epoch {i + 1} train_loss: {round(train_loss, 2)}, accuracy: {round(train_acc, 2)},'
              f'test_loss:{round(test_loss, 2)}, test_acc: {round(test_acc, 2)} ')



    metrics = {'train_acc_history': train_acc_history, 'train_loss_history': train_loss_history,
               'test_acc': test_acc_history, 'test_loss': test_loss_history}  # make dictionary of metrics
    cv_metrics.append(metrics)


Epoch 1 train_loss: 80.26, accuracy: 0.52,test_loss:4.83, test_acc: 0.53 
Epoch 2 train_loss: 79.06, accuracy: 0.52,test_loss:4.75, test_acc: 0.52 
Epoch 3 train_loss: 76.86, accuracy: 0.55,test_loss:4.55, test_acc: 0.61 
Epoch 4 train_loss: 73.5, accuracy: 0.63,test_loss:4.43, test_acc: 0.67 
Epoch 5 train_loss: 66.55, accuracy: 0.73,test_loss:4.43, test_acc: 0.68 
Epoch 6 train_loss: 58.3, accuracy: 0.79,test_loss:4.43, test_acc: 0.64 
Epoch 7 train_loss: 47.71, accuracy: 0.85,test_loss:4.83, test_acc: 0.64 
Epoch 8 train_loss: 42.25, accuracy: 0.88,test_loss:5.04, test_acc: 0.61 
Epoch 9 train_loss: 34.79, accuracy: 0.91,test_loss:5.2, test_acc: 0.65 
Epoch 10 train_loss: 32.64, accuracy: 0.91,test_loss:6.39, test_acc: 0.64 
Epoch 11 train_loss: 24.87, accuracy: 0.94,test_loss:6.77, test_acc: 0.58 
Epoch 12 train_loss: 20.28, accuracy: 0.95,test_loss:5.75, test_acc: 0.68 
Epoch 13 train_loss: 24.24, accuracy: 0.94,test_loss:6.49, test_acc: 0.64 
Epoch 14 train_loss: 20.12, accuracy:

In [ ]:
try:
    os.mkdir('model_evaluation')
except FileExistsError:
    pass

mod_eval_p = 'model_evaluation'# path to subdirectory for model evaluation storage

met_path = os.path.join(mod_eval_p, 'crnn5_cv_metrics.pickle') # path to pickle name for metric storage

with open(met_path, 'wb') as handle:
    pickle.dump(cv_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)